# Assignment 1

Deadline: 19.03.2025, 12:00 CET

<Add your name, student-id and emal address>

In [1]:
# Import standard libraries
import os
import sys
import timeit # To compute runtimes
from typing import Optional

# Import third-party libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import local modules
project_root = os.path.dirname(os.path.dirname(os.getcwd()))   # Change this path if needed
src_path = os.path.join(project_root, 'qpmwp-course\\src')
sys.path.append(project_root)
sys.path.append(src_path)
from estimation.covariance import Covariance
from estimation.expected_return import ExpectedReturn
from optimization.constraints import Constraints
from optimization.optimization import Optimization, Objective
from optimization.optimization_data import OptimizationData
from optimization.quadratic_program import QuadraticProgram, USABLE_SOLVERS

## 1. Solver horse race

### 1.a)
(3 points)

Generate a Multivariate-Normal random dataset of dimension TxN, T=1000, N=100, and compute a vector of expected returns, q, and a covariance matrix, P, using classes ExpectedReturn and Covariance respectively.

In [13]:

# Set the dimensions
T = 1000  # Number of time periods
N = 100   # Number of assets

# Generate a random mean vector from a normal distribution
mean = np.random.rand(N)
# Generate a random covariance matrix
A = np.random.rand(N, N)
cov = np.dot(A, A.transpose())  # To ensure the covariance matrix is positive semi-definite

# Generate the Multivariate-Normal random dataset
data = np.random.multivariate_normal(mean, cov, T)

# Convert the dataset to a DataFrame for easier manipulation
df = pd.DataFrame(data, columns=[f'Asset_{i+1}' for i in range(N)])


# Compute the vector of expected returns (mean returns) from df
scalefactor = 1
expected_return = ExpectedReturn(method='arithmetic', scalefactor=scalefactor)
expected_return.estimate(X=df, inplace=True)

# Compute the covariance matrix from df
covariance = Covariance(method='pearson')
covariance.estimate(X=df, inplace=True)


# Display the results
print("Vector of expected returns (q):")
print(q)
# 
print("\nCovariance matrix (P):")
print(P)

Vector of expected returns (q):
Asset_1      0.585686
Asset_2      0.289911
Asset_3      0.562792
Asset_4      0.267083
Asset_5      0.234132
               ...   
Asset_96     0.614382
Asset_97     0.393991
Asset_98     0.769293
Asset_99     0.611028
Asset_100    0.373884
Length: 100, dtype: float64

Covariance matrix (P):
             Asset_1    Asset_2    Asset_3    Asset_4    Asset_5    Asset_6  \
Asset_1    31.798509  24.828966  23.662610  25.713142  25.702657  26.527809   
Asset_2    24.828966  35.649954  25.004459  28.357161  27.693394  27.788563   
Asset_3    23.662610  25.004459  28.230190  23.935030  23.845259  25.355326   
Asset_4    25.713142  28.357161  23.935030  33.957776  25.772156  28.040510   
Asset_5    25.702657  27.693394  23.845259  25.772156  35.794538  29.065975   
...              ...        ...        ...        ...        ...        ...   
Asset_96   25.032439  26.960581  24.669273  25.286453  27.098781  28.033079   
Asset_97   25.846834  25.581775  24.009683

### 1.b)
(3 points)

Instantiate a constraints object by injecting column names of the data created in 1.a) as ids and add:
- a budget constaint (i.e., asset weights have to sum to one)
- lower bounds of 0.0 for all assets
- upper bounds of 0.2 for all assets
- group contraints such that the sum of the weights of the first 30 assets is <= 0.3, the sum of assets 31 to 60 is <= 0.4 and the sum of assets 61 to 100 is <= 0.5

In [14]:
# Instantiate the Constraints class
constraints = Constraints(ids = df.columns.tolist())

# Add budget constraint
constraints.add_budget(rhs=1, sense='=')

# Add box constraints (i.e., lower and upper bounds)
constraints.add_box(lower=0, upper=0.2)

# Add linear constraints
G = pd.DataFrame(np.zeros((3, N)), columns=constraints.ids)
G.iloc[0, :30] = 1
G.iloc[1, 30:60] = 1
G.iloc[2, 60:] = 1
h = pd.Series([0.3, 0.4, 0.5])
constraints.add_linear(G=G, sense='<=', rhs=h)


### 1.c) 
(4 points)

Solve a Mean-Variance optimization problem (using coefficients P and q in the objective function) which satisfies the above defined constraints.
Repeat the task for all open-source solvers in qpsolvers and compare the results in terms of:

- runtime
- accuracy: value of the primal problem.
- reliability: are all constarints fulfilled? Extract primal resisduals, dual residuals and duality gap.

Generate a DataFrame with the solvers as column names and the following row index: 'solution_found': bool, 'objective': float, 'primal_residual': float, 'dual_residual': float, 'duality_gap': float, 'runtime': float.

Put NA's for solvers that failed for some reason (e.g., unable to install the package or solvers throws an error during execution). 




In [33]:
list(USABLE_SOLVERS)

['highs', 'daqp', 'cvxopt', 'qpalm', 'quadprog', 'osqp']

In [79]:
import time
# Extract the constraints in the format required by the solver
GhAb = constraints.to_GhAb()

# Loop over solvers, instantiate the quadratic program, solve it and store the results
risk_aversion = 1

results = pd.DataFrame(columns=list(USABLE_SOLVERS), index={'solution_found': bool, 'objective': float, 'primal_residual': float, 'dual_residual': float, 'duality_gap': float, 'runtime': float})

for solver_name in USABLE_SOLVERS:
    qp = QuadraticProgram(
        P = covariance.matrix.to_numpy() * risk_aversion,
        q = expected_return.vector.to_numpy() * -1,
        G = GhAb['G'],
        h = GhAb['h'],
        A = GhAb['A'],
        b = GhAb['b'],
        lb = constraints.box['lower'].to_numpy(),
        ub = constraints.box['upper'].to_numpy(),
        solver = solver_name
    )

    start_time = time.time()
    qp.solve()
    end_time = time.time()

    sol = qp.results.get('solution')
    results.loc['solution_found', solver_name] = sol.found
    if sol.found:
        results.loc['objective', solver_name] = qp.objective_value()
        results.loc['primal_residual', solver_name] = sol.primal_residual()
        results.loc['dual_residual', solver_name] = sol.dual_residual()
        results.loc['duality_gap', solver_name] = sol.duality_gap()
        results.loc['runtime', solver_name] = end_time - start_time

Print and visualize the results

In [80]:
print(results)

                                  highs                   daqp  \
solution_found                     True                   True   
objective                      9.152325               9.152325   
primal_residual                     0.0                    0.0   
dual_residual                       0.0                    0.0   
duality_gap      1.2604335353216811e-08  8.846257060213247e-13   
runtime                        0.013607               0.003995   

                                cvxopt                  qpalm  \
solution_found                    True                   True   
objective                     9.152325               9.151026   
primal_residual                    0.0               0.000043   
dual_residual                      0.0                    0.0   
duality_gap      1.988092328781262e-07  0.0012989187257221638   
runtime                       0.056733               0.012029   

                               quadprog                 osqp  
solution_found    

## 2. Analytical Solution to Minimum-Variance Problem

(5 points)

- Create a `MinVariance` class that follows the structure of the `MeanVariance` class.
- Implement the `solve` method in `MinVariance` such that if `solver_name = 'analytical'`, the analytical solution is computed and stored within the object (if such a solution exists). If not, call the `solve` method from the parent class.
- Create a `Constraints` object by injecting the same ids as in part 1.b) and add a budget constraint.
- Instantiate a `MinVariance` object by setting `solver_name = 'analytical'` and passing instances of `Constraints` and `Covariance` as arguments.
- Create an `OptimizationData` object that contains an element `return_series`, which consists of the synthetic data generated in part 1.a).
- Solve the optimization problem using the created `MinVariance` object and compare the results to those obtained in part 1.c).


In [ ]:
# Define class MinVariance
class MinVariance(Optimization):

    def __init__(self,
                 constraints: Constraints,
                 covariance: Optional[Covariance] = None,
                 **kwargs):
        super().__init__(
            constraints=constraints,
            **kwargs
        )
        self.covariance = Covariance() if covariance is None else covariance

    def set_objective(self, optimization_data: OptimizationData) -> None:
        #<your code here>

    def solve(self) -> None:
        if self.params.get('solver_name') == 'analytical':
            #<your code here>
            return None
        else:
            return super().solve()


# Create a constraints object with just a budget constraint
#<your code here>

# Instantiate the MinVariance class
#<your code here>

# Prepare the optimization data and prepare the optimization problem
#<your code here>

# Solve the optimization problem and print the weights
#<your code here>